In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
import src.model as modelfunc
import src.sequencing as sequencing
from config import config
import pickle
import src.labeler as labeler
import src.anomaly_detection as anomaly


In [7]:
EPOCHS = 5

In [8]:
print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Details:", tf.config.list_physical_devices('GPU'))


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("Running on GPU")
else:
    print("GPU not available. Running on CPU")

TensorFlow Version: 2.18.0
Is TensorFlow built with CUDA? True
Num GPUs Available: 1
GPU Details: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1
Running on GPU


## parse GPMF

In [9]:
import telemetry_parser
import pandas as pd

def parse_telemetry(file_path):
    # Convert PosixPath to string if necessary
    tp = telemetry_parser.Parser(str(file_path))
    print(f"Camera: {tp.camera}, Model: {tp.model}")
    
    # Extract and normalize telemetry data
    data = tp.normalized_imu()
    df = pd.DataFrame(data)
    
    # Expand and organize columns
    df[['TIMESTAMP']] = pd.DataFrame((df['timestamp_ms'] / 1000).tolist(), index=df.index)
    df[['GYRO_x', 'GYRO_y', 'GYRO_z']] = pd.DataFrame(df['gyro'].tolist(), index=df.index)
    df[['ACCL_x', 'ACCL_y', 'ACCL_z']] = pd.DataFrame(df['accl'].tolist(), index=df.index)
    df.drop(columns=['timestamp_ms', 'gyro', 'accl'], inplace=True)
    df = df[['TIMESTAMP', 'GYRO_x', 'GYRO_y', 'GYRO_z', 'ACCL_x', 'ACCL_y', 'ACCL_z']]
    
    return df

# Example usage
df = parse_telemetry(config.DATA_DIR / "GH010041.MP4")
display(df)


Camera: GoPro, Model: GoPro Max


,TIMESTAMP,GYRO_x,GYRO_y,GYRO_z,ACCL_x,ACCL_y,ACCL_z
0,0.000000,2.074608,-1.159340,-4.027179,10.127098,0.901679,0.561151
1,0.005079,1.769518,-0.610179,-3.355983,10.424460,0.489209,0.522782
2,0.010158,1.647482,-0.671197,-4.942447,10.333333,0.956835,0.393285
3,0.015237,1.220357,-1.098322,-3.722090,10.172662,0.954436,0.019185
4,0.020316,0.793232,-1.220357,-2.318679,10.079137,0.127098,-0.395683
...,...,...,...,...,...,...,...
104649,531.507587,-10.006931,-11.898485,59.919548,11.587530,-13.803357,-1.134293
104650,531.512666,-2.867840,-15.132432,-16.596861,6.935252,-8.314149,-9.199041
104651,531.517745,-0.244071,-5.979751,-63.336549,0.935252,14.904077,-5.155875
104652,531.522824,-11.166270,0.549161,0.244071,3.760192,9.726619,-4.673861


In [14]:
labeled_dataframe_path = "data/GH010041_anonymized_labeled.csv"
df_labeled_frames = pd.read_csv(labeled_dataframe_path)

video_path = "data/GH010041.MP4"

#df_labeled_frames = pd.DataFrame(dict_list)
df = labeler.runner(video_path, df, df_labeled_frames)

display(df)

KeyError: 'frame_start'

In [15]:
df = pd.read_csv(config.DATA_DIR / "test_imu_frameindex_label_data.csv")  # Replace with your data file
overlap = 0.0
length = 10
num_actions = 5  # Example number of actions
df.dropna(inplace=True)

display(df)

# Convert LABEL to categorical and save mappings
df["LABEL"] = df["LABEL"].astype("category")
label_mapping = dict(enumerate(df["LABEL"].cat.categories))
df["LABEL"] = df["LABEL"].cat.codes

display(df)
print(label_mapping)

,TIMESTAMP,ACCL_x,ACCL_y,ACCL_z,GYRO_x,GYRO_y,GYRO_z,FRAME_INDEX,LABEL
0,0.000000,-2020.0,-2646.0,2432.0,1.0,-14.0,-13.0,1.0,opstappen
1,0.005102,-1845.0,-2600.0,2430.0,-6.0,-22.0,-6.0,1.0,opstappen
2,0.010204,-1966.0,-2623.0,2395.0,-9.0,-29.0,31.0,1.0,opstappen
3,0.015306,-2227.0,-2658.0,2398.0,-7.0,-32.0,54.0,1.0,opstappen
4,0.020408,-2275.0,-2636.0,2433.0,-6.0,-38.0,32.0,2.0,opstappen
...,...,...,...,...,...,...,...,...,...
29287,148.819444,-2279.0,-3173.0,1464.0,-89.0,-4.0,-24.0,8921.0,parkeren
29288,148.826389,-2210.0,-3170.0,1392.0,-81.0,-1.0,-36.0,8921.0,parkeren
29289,148.833333,-2119.0,-3153.0,1334.0,-61.0,-3.0,-44.0,8922.0,parkeren
29290,148.840278,-2124.0,-3150.0,1363.0,-39.0,-14.0,-35.0,8922.0,parkeren


,TIMESTAMP,ACCL_x,ACCL_y,ACCL_z,GYRO_x,GYRO_y,GYRO_z,FRAME_INDEX,LABEL
0,0.000000,-2020.0,-2646.0,2432.0,1.0,-14.0,-13.0,1.0,1
1,0.005102,-1845.0,-2600.0,2430.0,-6.0,-22.0,-6.0,1.0,1
2,0.010204,-1966.0,-2623.0,2395.0,-9.0,-29.0,31.0,1.0,1
3,0.015306,-2227.0,-2658.0,2398.0,-7.0,-32.0,54.0,1.0,1
4,0.020408,-2275.0,-2636.0,2433.0,-6.0,-38.0,32.0,2.0,1
...,...,...,...,...,...,...,...,...,...
29287,148.819444,-2279.0,-3173.0,1464.0,-89.0,-4.0,-24.0,8921.0,2
29288,148.826389,-2210.0,-3170.0,1392.0,-81.0,-1.0,-36.0,8921.0,2
29289,148.833333,-2119.0,-3153.0,1334.0,-61.0,-3.0,-44.0,8922.0,2
29290,148.840278,-2124.0,-3150.0,1363.0,-39.0,-14.0,-35.0,8922.0,2


{0: 'links', 1: 'opstappen', 2: 'parkeren', 3: 'rechtdoor', 4: 'rechts', 5: 'remmen'}


## Load data

In [16]:
# Preprocess data
# padded_sequences, padded_labels = sequencing.get_sequences_and_labels(df, overlap, length, num_actions)
sequences = sequencing.create_sequence(df, overlap, length)
padded_sequences = sequencing.get_sequences_pure_data(sequences)
padded_labels = sequencing.get_pure_labels(sequences)
padded_labels = tf.one_hot(padded_labels, depth=5)

I0000 00:00:1737551413.029842 1768588 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9702 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:26:00.0, compute capability: 6.1


In [17]:
print(len(padded_sequences)*len(padded_sequences[0]))
print(padded_sequences.shape)

29565
(15, 1971, 6)


In [18]:
validating = False

if validating:
    train_size = int(0.5 * len(padded_sequences))
    padded_sequences, X_val = padded_sequences[:train_size], padded_sequences[train_size:]
    padded_labels, y_val = padded_labels[:train_size], padded_labels[train_size:]

    print("Train shape:", padded_sequences.shape, padded_labels.shape)
    print("Validation shape:", X_val.shape, y_val.shape)
else:
    X_val = None
    y_val = None

In [19]:
# Generate sample weights to ignore padded timesteps
sample_weights = np.array([
    [1 if np.any(timestep != 0) else 0 for timestep in sequence]
    for sequence in padded_sequences
])

## Build Model

In [20]:
# Model parameters
print(padded_sequences.shape)
timesteps = padded_sequences.shape[1] # Variable-length sequences
features = 6  # IMU features (e.g., ax, ay, az, gx, gy, gz)
num_classes = 5  # Actions (e.g., left turn, right turn, stopping)
print(timesteps)
# Build the model
model = modelfunc.build_seq2seq_lstm((timesteps, features), num_classes, dropout=0.4)
model.summary()

(15, 1971, 6)
1971


/home/raf/interactive-telemetry-for-design/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1971, 256)      │       269,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1971, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1971, 128)      │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1971, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 1971, 64)       │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 1971, 5)        │           325 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 516,165 (1.97 MB)

 Trainable params: 516,165 (1.97 MB)

 Non-trainable params: 0 (0.00 B)

## Train Model

In [21]:
# _ = model.predict(X_train[:1])  # Perform a forward pass to initialize the model

# # Build the model explicitly
# model.build(input_shape=(None, timesteps, 6))  # Example: batch size = None, 50 timesteps, 6 features
# model.make_train_function()
# model.make_predict_function()

In [23]:
# # Example call to the function
history = modelfunc.train_model(
    model=model,  # Your pre-defined Keras model
    X_train=padded_sequences, 
    y_train=padded_labels,
    sample_weight=sample_weights,  # Optional; pass None if not using sample weights
    batch_size=16,  # Optional; defaults to 16
    epochs=EPOCHS       # Optional; defaults to 10
)

# # Example call to the function
# history = modelfunc.train_model(
#     model=model,  # Your pre-defined Keras model
#     X_train=padded_sequences, 
#     y_train=padded_labels,
#     X_val=X_val,
#     y_val=y_val,
#     sample_weight=sample_weights,  # Optional; pass None if not using sample weights
#     batch_size=16,  # Optional; defaults to 16
#     epochs=10       # Optional; defaults to 10
# )

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7444 - loss: 0.6687
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7641 - loss: 0.5938
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8077 - loss: 0.5376
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8316 - loss: 0.4954
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8385 - loss: 0.4691


## Model prediction

In [29]:
# Make predictions
predictions = model.predict(padded_sequences)

confidence_scores = anomaly.calculate_confidence(predictions)
predicted_classes = np.argmax(predictions, axis=-1)  # Class with the highest probability for each timestep

# Print results
print("Confidence scores shape:", confidence_scores.shape)  # Same shape as the input sequence
print("Predicted classes shape:", predicted_classes.shape)
print(predicted_classes)

display(predicted_classes)
print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
Confidence scores shape: (15, 1971)
Predicted classes shape: (15, 1971)
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 3 3 3]
 [1 1 1 ... 3 3 3]
 ...
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 2 2 2]
 [3 3 3 ... 3 3 3]]


array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 3, 3, 3],
       [1, 1, 1, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 2, 2, 2],
       [3, 3, 3, ..., 3, 3, 3]])

[[[0.1887   0.2188   0.1932   0.2109   0.1884  ]
  [0.1711   0.2576   0.1802   0.2217   0.1693  ]
  [0.1497   0.3135   0.1616   0.2286   0.1466  ]
  ...
  [0.06064  0.6597   0.0314   0.1907   0.05762 ]
  [0.06122  0.661    0.0323   0.1876   0.05786 ]
  [0.06238  0.653    0.0332   0.192    0.05963 ]]

 [[0.1901   0.2189   0.1875   0.213    0.1906  ]
  [0.1749   0.26     0.167    0.2253   0.1729  ]
  [0.1587   0.315    0.1422   0.2333   0.1509  ]
  ...
  [0.0651   0.17     0.06223  0.667    0.03552 ]
  [0.06805  0.1685   0.06537  0.6616   0.03653 ]
  [0.0702   0.1788   0.07367  0.641    0.0364  ]]

 [[0.1915   0.2079   0.2054   0.2075   0.1876  ]
  [0.1826   0.2236   0.2092   0.2162   0.1683  ]
  [0.1719   0.2471   0.2108   0.2241   0.1461  ]
  ...
  [0.01459  0.01447  0.00546  0.929    0.0364  ]
  [0.013985 0.01424  0.005447 0.9287   0.0375  ]
  [0.014084 0.014175 0.005337 0.9277   0.03876 ]]

 ...

 [[0.1877   0.1864   0.1805   0.254    0.1914  ]
  [0.163    0.1576   0.1416   0.3638   

In [32]:
# Flatten the 3D array to 2D (e.g., concatenate along the first axis)
flattened_data = predictions.reshape(-1, predictions.shape[-1])

# Save to a CSV file
np.savetxt("output.csv", flattened_data, delimiter=",", fmt="%.5f")

# predictions_file = config.DATA_DIR / 'model_predictions.pkl'

# with predictions_file.open('wb') as file:
#     pickle.dump(predictions, file)

In [ ]:
# should_load_predictions = False

# if should_load_predictions:
#     with predictions_file.open('rb') as file:
#         predictions = pickle.load(file)

In [ ]:
print(predicted_classes)

In [ ]:
print(len(predicted_classes.flatten()))
bound = len(predicted_classes.flatten())
bound2 = print(len(df["TIMESTAMP"]))

df_results = pd.DataFrame({
    "TIMESTAMP": df["TIMESTAMP"].values[:bound],  # Pas lengte aan indien nodig
    "FRAME_INDEX": df["FRAME_INDEX"].values[:bound],
    "predicted_class": predicted_classes.flatten(),
    "confidence": confidence_scores.flatten()
})

df_results["predicted_class"] = df_results["predicted_class"].map(lambda x: label_mapping[x])

print(df_results.head(20))

In [ ]:
# Convert DataFrame to list of dictionaries based on changes in predicted_class

def df_to_dict(df):
    result = []
    start_idx = 0  # Track the start index of the current predicted_class

    for i in range(1, len(df)):
        if df.loc[i, 'predicted_class'] != df.loc[i - 1, 'predicted_class']:
            # Add entry when predicted_class changes
            result.append({
                "label": df.loc[start_idx, 'predicted_class'],
                "frame_start": df.loc[start_idx, 'FRAME_INDEX'],
                "frame_end": df.loc[i - 1, 'FRAME_INDEX']
            })
            start_idx = i  # Update start index for next segment

    # Add the last segment
    result.append({
        "label": df.loc[start_idx, 'predicted_class'],
        "frame_start": df.loc[start_idx, 'FRAME_INDEX'],
        "frame_end": df.loc[len(df) - 1, 'FRAME_INDEX']
    })

    return df


result = df_to_dict(df_results)
print(result)

In [ ]:
def dict_to_labeledframes(dict_list):
    
    df_label = pd.DataFrame(dict_list)

    return df_label

dict_to_labeledframes(result)